# 5. ФУНКЦИИ ОБОГАЩЕНИЯ ДАННЫХ

In [ ]:
# 1. ИМПОРТ БИБЛИОТЕК
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import os

# Визуализация
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
from plotly.offline import plot

# Машинное обучение (для кластеризации в RFM)
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer
from sklearn.preprocessing import FunctionTransformer

In [ ]:
'''1. Загрузка данных'''
data = pd.read_csv('merged_sales_data.csv')

4. RFM-анализ

In [ ]:
# RFM-анализ

current_date = merged_data['OrderDate'].max()

rfm = merged_data.groupby('CustomerKey').agg({
    'OrderDate': lambda x: (current_date - x.max()).days,  # Recency
    'SalesOrderNumber': 'nunique',                          # Frequency
    'SalesAmount': 'sum'                                    # Monetary
}).rename(columns={
    'OrderDate': 'Recency',
    'SalesOrderNumber': 'Frequency',
    'SalesAmount': 'Monetary'
})
# Квантили для Recency и Monetary
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=4, labels=[4,3,2,1], duplicates='drop')
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=4, labels=[1,2,3,4], duplicates='drop')
# Для Frequency используем cut (равные интервалы), если qcut не работает
try:
    rfm['F_Score'] = pd.qcut(rfm['Frequency'], q=4, labels=[1,2,3,4], duplicates='drop')
except ValueError:
    print("qcut не работает для Frequency — используем cut (равные интервалы)")
    rfm['F_Score'] = pd.cut(rfm['Frequency'], bins=4, labels=[1,2,3,4], duplicates='drop')

# RFM Score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
# Сегментация
def assign_rfm_segment(score):
    if score == '444':
        return 'Champions'
    elif score.startswith('4'):
        return 'Loyal Customers'
    elif score.endswith('4'):
        return 'Big Spenders'
    elif score.startswith('1'):
        return 'Lost Customers'
    else:
        return 'Potential'

rfm['Segment'] = rfm['RFM_Score'].apply(assign_rfm_segment)

print("\nRFM сегментация:")
print(rfm['Segment'].value_counts())

In [ ]:
# 🎯 RFM-АНАЛИЗ КЛИЕНТОВ
print("🎯 RFM-АНАЛИЗ КЛИЕНТОВ")
print("=" * 50)

# Расчет RFM метрик
print("📊 РАСЧЕТ RFM МЕТРИК:")
snapshot_date = df['OrderDate'].max() + pd.Timedelta(days=1)
print(f"📅 Дата анализа: {snapshot_date.date()}")

rfm = df.groupby('CustomerKey').agg({
    'OrderDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'SalesOrderNumber': 'nunique',  # Frequency
    'SalesAmount': 'sum'            # Monetary
}).reset_index()

rfm.columns = ['CustomerKey', 'Recency', 'Frequency', 'Monetary']

print(f"✅ RFM метрики рассчитаны для {len(rfm)} клиентов")
print(f"   • Recency (давность): дней с последнего заказа")
print(f"   • Frequency (частота): количество заказов") 
print(f"   • Monetary (деньги): общая выручка")

# Основные статистики RFM
print("\n📊 ОСНОВНЫЕ СТАТИСТИКИ RFM:")
display(rfm[['Recency', 'Frequency', 'Monetary']].describe())

# Создание RFM квантилей
print("🎯 СОЗДАНИЕ RFM КВАНТИЛЕЙ:")
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

print("✅ RFM квантили созданы:")
print(f"   • R_Score (Recency): 5 (самые новые) - 1 (самые старые)")
print(f"   • F_Score (Frequency): 1 (редкие) - 5 (частые)")
print(f"   • M_Score (Monetary): 1 (малые суммы) - 5 (крупные суммы)")

# RFM сегментация
print("\n🎯 RFM СЕГМЕНТАЦИЯ КЛИЕНТОВ:")
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Упрощенная сегментация
segment_results = []
for index, row in rfm.iterrows():
    r, f, m = int(row['R_Score']), int(row['F_Score']), int(row['M_Score'])
    
    if r >= 4 and f >= 4 and m >= 4:
        segment = 'Champions'
    elif r >= 3 and f >= 3 and m >= 3:
        segment = 'Loyal Customers'
    elif r >= 4 and f <= 2 and m <= 2:
        segment = 'New Customers'
    elif r >= 3 and f <= 2 and m <= 2:
        segment = 'Promising'
    elif r <= 2 and f >= 3 and m >= 3:
        segment = 'Need Attention'
    elif r <= 2 and f <= 2 and m <= 2:
        segment = 'Hibernating'
    else:
        segment = 'Regular'
    
    segment_results.append(segment)

rfm['Segment'] = segment_results

print("✅ Сегментация завершена:")
print("   • Champions: самые ценные клиенты")
print("   • Loyal Customers: лояльные клиенты") 
print("   • New Customers: новые клиенты")
print("   • Promising: перспективные клиенты")
print("   • Need Attention: требуют внимания")
print("   • Hibernating: спящие клиенты")
print("   • Regular: обычные клиенты")

# Визуализация RFM
print("\n📈 ВИЗУАЛИЗАЦИЯ RFM-АНАЛИЗА:")
fig = px.scatter(rfm, x='Recency', y='Monetary', color='Segment',
                size='Frequency', hover_data=['CustomerKey'],
                title='🎯 RFM-анализ клиентов',
                labels={'Recency': 'Дней с последнего заказа (Recency)',
                       'Monetary': 'Общая выручка (Monetary)',
                       'Frequency': 'Частота заказов',
                       'Segment': 'Сегмент'},
                color_discrete_map={
                    'Champions': '#00FF00',
                    'Loyal Customers': '#32CD32', 
                    'New Customers': '#1E90FF',
                    'Promising': '#FFD700',
                    'Need Attention': '#FFA500',
                    'Hibernating': '#A9A9A9',
                    'Regular': '#808080'
                })
fig.show()
print("✅ Интерактивная визуализация RFM создана")

# Распределение сегментов
print("\n📊 РАСПРЕДЕЛЕНИЕ КЛИЕНТОВ ПО СЕГМЕНТАМ:")
segment_distribution = rfm['Segment'].value_counts()

total_clients = len(rfm)
print(f"👥 Всего клиентов: {total_clients}")

for segment, count in segment_distribution.items():
    percentage = (count / total_clients) * 100
    revenue_share = (rfm[rfm['Segment'] == segment]['Monetary'].sum() / rfm['Monetary'].sum()) * 100
    print(f"   • {segment}: {count} клиентов ({percentage:.1f}%) - {revenue_share:.1f}% выручки")

# Визуализация распределения сегментов
print("\n📊 ВИЗУАЛИЗАЦИЯ РАСПРЕДЕЛЕНИЯ СЕГМЕНТОВ:")
fig_pie = px.pie(segment_distribution, 
                 values=segment_distribution.values, 
                 names=segment_distribution.index,
                 title='📊 Распределение клиентов по RFM-сегментам',
                 color_discrete_sequence=px.colors.qualitative.Set3)
fig_pie.show()
print("✅ Круговая диаграмма распределения сегментов создана")

# Анализ лучших клиентов
print("\n🏆 АНАЛИЗ ЛУЧШИХ КЛИЕНТОВ:")
champions = rfm[rfm['Segment'] == 'Champions']
if len(champions) > 0:
    print(f"   • Champions: {len(champions)} клиентов")
    print(f"   • Средняя выручка на Champion: ${champions['Monetary'].mean():,.0f}")
    print(f"   • Общая выручка Champions: ${champions['Monetary'].sum():,.0f}")
    print(f"   • Доля выручки Champions: {(champions['Monetary'].sum() / rfm['Monetary'].sum())*100:.1f}%")

# Сохранение результатов
print(f"\n💾 РЕЗУЛЬТАТЫ RFM-АНАЛИЗА:")
print(f"   • Проанализировано клиентов: {len(rfm)}")
print(f"   • Всего сегментов: {len(segment_distribution)}")
print(f"   • Самый крупный сегмент: {segment_distribution.index[0]} ({segment_distribution.iloc[0]} клиентов)")
print(f"   • Общая выручка всех клиентов: ${rfm['Monetary'].sum():,.0f}")

print("\n👀 ПРЕВЬЮ RFM ДАННЫХ:")
display(rfm.head(10))

print("✅ RFM-АНАЛИЗ УСПЕШНО ЗАВЕРШЕН!")

In [ ]:
# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

5. ABC-анализ

In [ ]:
# - ABC

abc = merged_data.groupby('ProductName')['SalesAmount'].sum().sort_values(ascending=False)
abc_pct = abc.cumsum() / abc.sum() * 100

abc_df = pd.DataFrame({
    'ProductName': abc.index,
    'SalesAmount': abc.values,
    'CumulativePct': abc_pct.values
})

abc_df['ABC_Category'] = pd.cut(abc_df['CumulativePct'], bins=[0, 80, 95, 100], labels=['A', 'B', 'C'])

print("\nABC-анализ (по продуктам):")
print(abc_df.head(10))
# ABC по регионам
abc_by_region = abc_df.merge(merged_data[['ProductName', 'Region']], on='ProductName', how='left')
abc_by_region_summary = abc_by_region.groupby('Region')['ABC_Category'].value_counts().unstack(fill_value=0)
print("\nABC-анализ по регионам:")
print(abc_by_region_summary)

In [ ]:
# -XYZ-анализ (по стабильности продаж)

xyz = merged_data.groupby(['ProductName', 'OrderYear', 'OrderMonth'])['SalesAmount'].sum().reset_index()

xyz_std = xyz.groupby('ProductName')['SalesAmount'].std().fillna(0).reset_index()
xyz_mean = xyz.groupby('ProductName')['SalesAmount'].mean().reset_index()

xyz_combined = xyz_std.merge(xyz_mean, on='ProductName', suffixes=('_std', '_mean'))
xyz_combined['CV'] = xyz_combined['SalesAmount_std'] / xyz_combined['SalesAmount_mean']
xyz_combined['CV'] = xyz_combined['CV'].replace([np.inf, -np.inf], np.nan).fillna(0)
# Категоризация XYZ
bins = [0, 0.1, 0.25, 1]
labels = ['X', 'Y', 'Z']
xyz_combined['XYZ_Category'] = pd.cut(xyz_combined['CV'], bins=bins, labels=labels)

print("\nXYZ-анализ (по стабильности продаж):")
print(xyz_combined.sort_values('CV').head(10))

In [ ]:
# 📦 ABC-XYZ АНАЛИЗ ПРОДУКТОВ
print("📦 ABC-XYZ АНАЛИЗ ПРОДУКТОВ")
print("=" * 50)

# ABC анализ (по выручке)
print("📊 ABC АНАЛИЗ (ПО ВЫРУЧКЕ):")
product_sales = df.groupby(['ProductKey', 'ProductName']).agg({
    'SalesAmount': 'sum',
    'OrderQuantity': 'sum',
    'Profit': 'sum',
    'SalesOrderNumber': 'nunique'
}).reset_index()

print(f"✅ Проанализировано продуктов: {len(product_sales)}")
print(f"💰 Общая выручка от продуктов: ${product_sales['SalesAmount'].sum():,.0f}")

# Сортировка по выручке
product_sales = product_sales.sort_values('SalesAmount', ascending=False)
product_sales['CumulativePercentage'] = (product_sales['SalesAmount'].cumsum() / 
                                       product_sales['SalesAmount'].sum() * 100)

print("📈 РАСПРЕДЕЛЕНИЕ ВЫРУЧКИ ПО ПРОДУКТАМ:")
print(f"   • Топ-1 продукт: {product_sales.iloc[0]['ProductName']} (${product_sales.iloc[0]['SalesAmount']:,.0f})")
print(f"   • Топ-5 продуктов: {product_sales['SalesAmount'].head(5).sum():,.0f} $ ({product_sales['CumulativePercentage'].iloc[4]:.1f}%)")
print(f"   • Топ-10 продуктов: {product_sales['SalesAmount'].head(10).sum():,.0f} $ ({product_sales['CumulativePercentage'].iloc[9]:.1f}%)")

# ABC категории
print("\n🎯 НАЗНАЧЕНИЕ ABC КАТЕГОРИЙ:")
abc_categories = []
for percentage in product_sales['CumulativePercentage']:
    if percentage <= 80:
        abc_categories.append('A')
    elif percentage <= 95:
        abc_categories.append('B')
    else:
        abc_categories.append('C')

product_sales['ABC_Category'] = abc_categories

print("✅ ABC категории назначены:")
print("   • A: топ 80% выру чки (ключевые продукты)")
print("   • B: следующие 15% выручки (важные продукты)") 
print("   • C: оставшиеся 5% выручки (второстепенные продукты)")

# XYZ анализ (по стабильности продаж)
print("\n📊 XYZ АНАЛИЗ (ПО СТАБИЛЬНОСТИ ПРОДАЖ):")
try:
    monthly_product_sales = df.pivot_table(
        index='ProductKey', 
        columns=df['OrderDate'].dt.to_period('M'), 
        values='OrderQuantity', 
        aggfunc='sum'
    ).fillna(0)
    
    print(f"✅ Собраны месячные данные по {len(monthly_product_sales)} продуктам")
    print(f"📅 Период анализа: {len(monthly_product_sales.columns)} месяцев")
    
    # Коэффициент вариации
    cv = monthly_product_sales.std(axis=1) / (monthly_product_sales.mean(axis=1) + 0.0001)
    
    product_sales = product_sales.merge(cv.rename('CV'), left_on='ProductKey', right_index=True)
    
    # XYZ категории
    xyz_categories = []
    for cv_value in product_sales['CV']:
        if cv_value <= 0.3:
            xyz_categories.append('X')
        elif cv_value <= 0.6:
            xyz_categories.append('Y')
        else:
            xyz_categories.append('Z')
    
    product_sales['XYZ_Category'] = xyz_categories
    product_sales['ABC_XYZ'] = product_sales['ABC_Category'] + '-' + product_sales['XYZ_Category']
    
    print("✅ XYZ категории назначены:")
    print("   • X: стабильные продажи (CV ≤ 0.3)")
    print("   • Y: переменные продажи (0.3 < CV ≤ 0.6)")
    print("   • Z: нестабильные продажи (CV > 0.6)")
    
except Exception as e:
    print(f"❌ XYZ анализ не удался: {e}")
    product_sales['XYZ_Category'] = 'Unknown'
    product_sales['ABC_XYZ'] = product_sales['ABC_Category'] + '-Unknown'
    print("⚠️  Используется только ABC анализ")

# Визуализация ABC-XYZ матрицы
print("\n📈 ВИЗУАЛИЗАЦИЯ ABC-XYZ МАТРИЦЫ:")
if 'CV' in product_sales.columns:
    fig = px.scatter(product_sales, x='CV', y='SalesAmount', color='ABC_XYZ',
                    size='OrderQuantity', hover_data=['ProductName'],
                    title='📦 ABC-XYZ АНАЛИЗ ПРОДУКТОВ',
                    labels={'CV': 'Коэффициент вариации (стабильность)',
                           'SalesAmount': 'Выручка ($)',
                           'OrderQuantity': 'Количество проданных единиц',
                           'ABC_XYZ': 'Категория'},
                    color_discrete_map={
                        'A-X': '#00FF00', 'A-Y': '#90EE90', 'A-Z': '#ADFF2F',
                        'B-X': '#1E90FF', 'B-Y': '#87CEEB', 'B-Z': '#ADD8E6', 
                        'C-X': '#FFA500', 'C-Y': '#FFD700', 'C-Z': '#FFFF00'
                    })
    fig.show()
    print("✅ Интерактивная матрица ABC-XYZ создана")
else:
    print("❌ Визуализация невозможна: отсутствуют данные CV")

# Анализ результатов
print("\n📊 РАСПРЕДЕЛЕНИЕ ABC-КАТЕГОРИЙ:")
abc_counts = product_sales['ABC_Category'].value_counts()
total_products = len(product_sales)
total_revenue = product_sales['SalesAmount'].sum()

for category, count in abc_counts.items():
    percentage = (count / total_products) * 100
    category_revenue = product_sales[product_sales['ABC_Category'] == category]['SalesAmount'].sum()
    revenue_share = (category_revenue / total_revenue) * 100
    avg_revenue_per_product = category_revenue / count
    
    print(f"   • {category}: {count} продуктов ({percentage:.1f}%)")
    print(f"     Выручка: ${category_revenue:,.0f} ({revenue_share:.1f}%)")
    print(f"     Средняя выручка на продукт: ${avg_revenue_per_product:,.0f}")

if 'XYZ_Category' in product_sales.columns and 'Unknown' not in product_sales['XYZ_Category'].unique():
    print("\n📊 РАСПРЕДЕЛЕНИЕ XYZ-КАТЕГОРИЙ:")
    xyz_counts = product_sales['XYZ_Category'].value_counts()
    
    for category, count in xyz_counts.items():
        percentage = (count / total_products) * 100
        category_cv_avg = product_sales[product_sales['XYZ_Category'] == category]['CV'].mean()
        print(f"   • {category}: {count} продуктов ({percentage:.1f}%)")
        print(f"     Средний CV: {category_cv_avg:.3f}")

# Анализ ключевых сегментов
print("\n🏆 АНАЛИЗ КЛЮЧЕВЫХ СЕГМЕНТОВ:")
if 'ABC_XYZ' in product_sales.columns:
    key_segments = ['A-X', 'A-Y', 'B-X']
    for segment in key_segments:
        segment_data = product_sales[product_sales['ABC_XYZ'] == segment]
        if len(segment_data) > 0:
            segment_revenue = segment_data['SalesAmount'].sum()
            segment_share = (segment_revenue / total_revenue) * 100
            print(f"   • {segment}: {len(segment_data)} продуктов, {segment_share:.1f}% выручки")
            
            if len(segment_data) > 0:
                top_product = segment_data.iloc[0]
                print(f"     Топ продукт: {top_product['ProductName']} (${top_product['SalesAmount']:,.0f})")

# Рекомендации
print("\n💡 РЕКОМЕНДАЦИИ ПО ABC-XYZ АНАЛИЗУ:")
a_products = product_sales[product_sales['ABC_Category'] == 'A']
if len(a_products) > 0:
    print(f"   • Фокус на {len(a_products)} продуктах категории A ({a_products['SalesAmount'].sum()/total_revenue*100:.1f}% выручки)")
    
if 'XYZ_Category' in product_sales.columns:
    unstable_products = product_sales[product_sales['XYZ_Category'] == 'Z']
    if len(unstable_products) > 0:
        print(f"   • {len(unstable_products)} продуктов с нестабильными продажами (категория Z)")

print(f"\n📈 ИТОГИ ABC-XYZ АНАЛИЗА:")
print(f"   • Проанализировано продуктов: {len(product_sales)}")
print(f"   • Общая выручка: ${total_revenue:,.0f}")
print(f"   • ABC категорий: {len(abc_counts)}")
if 'XYZ_Category' in product_sales.columns:
    print(f"   • XYZ категорий: {len(xyz_counts) if 'xyz_counts' in locals() else 'N/A'}")

print("\n👀 ПРЕВЬЮ РЕЗУЛЬТАТОВ ABC-XYZ:")
display(product_sales[['ProductName', 'ABC_Category', 'XYZ_Category', 'ABC_XYZ', 'SalesAmount', 'OrderQuantity']].head(10))

print("✅ ABC-XYZ АНАЛИЗ ПРОДУКТОВ ЗАВЕРШЕН!")

In [ ]:
# RFM: Recency, Frequency, Monetary
current_date = merged_data['OrderDate'].max()

rfm = merged_data.groupby('CustomerKey').agg({
    'OrderDate': lambda x: (current_date - x.max()).days,  # Recency
    'SalesOrderNumber': 'nunique',                          # Frequency
    'SalesAmount': 'sum'                                    # Monetary
}).rename(columns={
    'OrderDate': 'Recency',
    'SalesOrderNumber': 'Frequency',
    'SalesAmount': 'Monetary'
})

# Квантили
rfm['R_Score'] = pd.qcut(rfm['Recency'], q=4, labels=[4,3,2,1])
rfm['F_Score'] = pd.qcut(rfm['Frequency'], q=4, labels=[1,2,3,4])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=4, labels=[1,2,3,4])

rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Сегментация
def assign_rfm_segment(score):
    if score == '444':
        return 'Champions'
    elif score.startswith('4'):
        return 'Loyal Customers'
    elif score.endswith('4'):
        return 'Big Spenders'
    elif score.startswith('1'):
        return 'Lost Customers'
    else:
        return 'Potential'

rfm['Segment'] = rfm['RFM_Score'].apply(assign_rfm_segment)

print("\nRFM сегментация:")
print(rfm['Segment'].value_counts())

# Сегментация с географией
rfm_with_geo = rfm.merge(customer[['CustomerKey', 'CustomerCountry']], left_index=True, right_on='CustomerKey', how='left')
rfm_with_geo = rfm_with_geo.merge(territories[['SalesTerritoryKey', 'Group']], 
                                  left_on='CustomerCountry', right_on='SalesTerritoryKey', how='left')

print("\nRFM по группе регионов:")
print(rfm_with_geo.groupby('Group')['Segment'].value_counts())
# ...

In [ ]:
# ABC-анализ по продуктам
abc = merged_data.groupby('ProductName')['SalesAmount'].sum().sort_values(ascending=False)
abc_pct = abc.cumsum() / abc.sum() * 100

abc_df = pd.DataFrame({
    'ProductName': abc.index,
    'SalesAmount': abc.values,
    'CumulativePct': abc_pct.values
})

abc_df['ABC_Category'] = pd.cut(abc_df['CumulativePct'], bins=[0, 80, 95, 100], labels=['A', 'B', 'C'])

print("\nABC-анализ (по продуктам):")
print(abc_df.head(10))

# ABC по регионам
abc_by_region = abc_df.merge(merged_data[['ProductName', 'Region']], on='ProductName', how='left')
abc_by_region_summary = abc_by_region.groupby('Region')['ABC_Category'].value_counts().unstack(fill_value=0)
print("\nABC-анализ по регионам:")
print(abc_by_region_summary)

In [ ]:
# Заключение
def generate_conclusion():
    conclusion = f"""
    🔍 АНАЛИТИЧЕСКОЕ ЗАКЛЮЧЕНИЕ:

    1. Данные успешно объединены и очищены.
    2. EDA показал:
       - Продажи в группе регионов 'Europe' и 'North America' самые высокие.
       - Клиенты из 'Australia' приносят высокий средний доход.
       - Регионы 'Europe' и 'North America' — основные драйверы выручки.
    3. RFM-анализ:
       - Champions: {len(rfm[rfm['Segment'] == 'Champions'])} клиентов — высокий потенциал.
       - Lost Customers: {len(rfm[rfm['Segment'] == 'Lost Customers'])} — требуют возврата.
    4. ABC-анализ:
       - A-продукты: {len(abc_df[abc_df['ABC_Category'] == 'A'])} позиций — фокусировать маркетинг.
    5. Инсайты:
       - Продажи по регионам сильно различаются.
       - Продукты A-категории популярны в 'Europe' и 'North America'.
       - Клиенты с высоким доходом предпочитают определённые категории.

    💡 Рекомендации:
    - Усилить маркетинг в 'Europe' и 'North America'.
    - Персонализировать предложения по RFM-сегментам.
    - Провести A/B тесты по продуктам C-категории.
    """
    print(conclusion)

generate_conclusion()